In [192]:
import numpy as np
import pandas as pd
import re
from pandas import DataFrame
from sqlalchemy import create_engine
from matplotlib import pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn import cross_validation
from sqlalchemy import create_engine
%matplotlib inline

In [23]:
# Connecting to the db
host_db = "cal-patent-lab.chhaitskv8dz.us-west-2.rds.amazonaws.com"
username = "***REMOVED***"
password = "***REMOVED***"
db = "***REMOVED***"

engine = create_engine("mysql://{}:{}@{}/{}".format(
    username, password, host_db, db))
connection = engine.connect()
patents_decision = pd.read_sql("SELECT patent_id, invalidated, CAST( claims_text AS CHAR ) FROM patents_decision", engine)
connection.close()

In [24]:
patents_data = patents_decision[patents_decision["invalidated"].isin([0, 1])]
patents_id = list(patents_data.patent_id)

In [25]:
with open("mysql.txt") as f:
    for line in f:
        connector = line

In [26]:
def get_information(query):
    # Connecting to the fung db
    engine_fung = create_engine(connector)
    connection_fung = engine_fung.connect()
    patent_info = pd.read_sql(
        query, engine_fung)
    # s =str("SELECT * FROM patent WHERE id IN {}".format)
    # rs = connection_fung.execute(s, patents_id=tuple(patents_id,)).fetchall()
    connection.close()
    return patent_info

In [151]:
query =str("SELECT * FROM patent WHERE id IN {}".format(tuple(patents_id)))
inventor_query = "SELECT * FROM patent JOIN patent_inventor \
ON patent.id = patent_inventor.patent_id JOIN inventor on patent_inventor.inventor_id=inventor.id \
WHERE patent.id IN {};".format(
tuple(patents_id))
lawyer_query = "SELECT patent.id AS patent_id, lawyer.organization FROM patent JOIN patent_lawyer ON patent.id = \
patent_lawyer.patent_id JOIN lawyer on patent_lawyer.lawyer_id=lawyer.id WHERE patent.id IN {};".format(
tuple(patents_id))
assignee_query ="SELECT patent.id AS patent_id, assignee.organization FROM patent JOIN patent_assignee ON patent.id = \
patent_assignee.patent_id JOIN assignee on patent_assignee.assignee_id=assignee.id WHERE patent.id IN {};".format(
tuple(patents_id))

inventor_info = get_information(inventor_query)
lawyer_info = get_information(lawyer_query)
lawyer_info = lawyer_info.rename(columns={"organization": "lawyer_organization"})
assignee_info = get_information(assignee_query)
assignee_info = assignee_info.rename(columns={"organization": "assignee_organization"})
inventor_info = inventor_info.drop(['number', 'id', 'inventor_id', 'nationality'], axis=1)
inventor_info = inventor_info.rename(columns={'name_first': 'inventor_name_first',
                              'name_last': 'inventor_name_last'})

In [152]:
# Merging all information in just one dataframe
temp_info = pd.merge(lawyer_info, assignee_info, how="outer", on="patent_id")
patent_info = pd.merge(inventor_info, temp_info, how="outer", on="patent_id")

# Getting the list of patents with a decision
patents_data = patents_decision[patents_decision["invalidated"].isin([0, 1])]

# full dataframe with information on the inventor, assignee and lawyer for each patent with a decision
patent_info_with_decision = pd.merge(patents_data, patent_info, how="inner", on="patent_id")
patent_info_with_decision = patent_info_with_decision.drop(['inventor_name_first', 'inventor_name_last'], axis=1)
patent_data_decision = patent_info_with_decision.drop_duplicates()

In [153]:
len(patent_data_decision)

1910

In [154]:
patent_data_decision.loc[0]

patent_id                                                                5004643
invalidated                                                                    0
CAST( claims_text AS CHAR )    What is claimed is: 1. 1. A flexible porous we...
type                                                                            
country                                                                         
date                                                                  1991-04-02
abstract                       An improved process is provided for treating a...
title                                    Silicone polymer-internally coated webs
kind                                                                            
num_claims                                                                    27
lawyer_organization                                                          NaN
assignee_organization                                             SILI-TEX, INC.
Name: 0, dtype: object

In [156]:
patent_data_decision = patent_data_decision.drop(["country", "type", "date", "title", "abstract", "kind"], axis=1)

In [185]:
patent_data_decision = patent_data_decision[patent_data_decision.lawyer_organization.notnull() & patent_data_decision.assignee_organization.notnull()
                    & ~patent_data_decision.lawyer_organization.isin(["", " "])]

In [204]:
X = patent_data_decision['CAST( claims_text AS CHAR )'].as_matrix()
y = patent_data_decision['invalidated'].as_matrix()

X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.2, random_state=20)

In [208]:
# this normalizes word frequency based on use in each patent vs. all patents

# Removing element with no text
index_to_remove= []
for i in range(len(X)):
    if X[i] == None:
        index_to_remove.append(i)
        
print(index_to_remove)
        
X = np.delete(X, index_to_remove)
y = np.delete(y, index_to_remove)

tfidf = TfidfVectorizer()
tfidf.fit(X[:])
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)

[]


In [210]:
X_train

<847x12847 sparse matrix of type '<type 'numpy.float64'>'
	with 131869 stored elements in Compressed Sparse Row format>

In [197]:
len(X)

1059

In [199]:
X[1,1]

'What is claimed: 1. 1. A stent joining means for joining a first endoluminal stent to a second endoluminal stent to define a continuous lumen through the first and second endoluminal stents, said stent joining means comprising: PA1 a male engaging portion on said first endoluminal stent which has an outer surface and can be compressed radially inwardly; and PA1 a female portion on said second endoluminal stent cooperating with said male engaging portion, said female portion having an inner surface; PA1 wherein said first endoluminal stent and said second endoluminal stent consist of a shape memory alloy and the male engaging portion can be entered into the female portion in a radially compressed state and thereafter thermally induced to expand in the female portion and wherein a frictional inter-engagement between said outer surface of the male engaging portion and said inner surface of the female portion prevents longitudinal movement of the first endoluminal stent relative to the se